In [65]:
%pip install --upgrade google-genai python-dotenv --quiet
# google-genai is Google’s GenAI Python SDK that talks to Gemini and the embedding models.


Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install python-dotenv --quiet
# python-dotenv is a library that loads environment variables from a .env file into the environment of the current process.

Note: you may need to restart the kernel to use updated packages.


In [67]:
# Imports 
from google import genai
from google.genai import types #helper types for safety
# we will use the dotenv library to load the environment variables from the .env file.
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise RuntimeError("GEMINI_API_KEY is not set")

client = genai.Client(api_key=API_KEY)

In [68]:
prompt_text = "I like cake"

# Counting Tokens first. 
# Counting tokens helps you avoid exceeding model context limits and estimate cost.
# We call the SDK's token counting method which uses the model's tokenizer.

token_count = client.models.count_tokens(
    model = "gemini-2.5-flash",
    contents=[types.Content(parts=[types.Part(text=prompt_text)])]

)

print("Total Token Count :",getattr(token_count,"total_tokens",None))
# gemini doesn't allow you to see the tokens generated by the model. 




Total Token Count : 3


In [69]:
# Create an internal embedding for the prompt
embed_response = client.models.embed_content(
    model="gemini-embedding-001",                   
    contents=[prompt_text],                         
    config=types.EmbedContentConfig(
        task_type="SEMANTIC_SIMILARITY",            
        output_dimensionality=3072                 
    ),
)

embeddings = embed_response.embeddings[0].values
print("Number of Embeddings created:" ,len(embeddings))



Number of Embeddings created: 3072


In [70]:
# Basic safety / moderation pre-check

# You can instruct Gemini to use stricter safety thresholds for the request.
# Here we set two categories to block low+ probability content (example).
# If the input is disallowed the API may block the response or provide safety metadata.
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

In [71]:
# This `GenerateContentConfig` will be passed to the generator below.
gen_config = types.GenerateContentConfig(
    safety_settings=safety_settings,
    # system_instruction can set the assistant behavior (optional)
    system_instruction="You are a friendly assistant that replies in a very simple way and as if you're talking to a 10 year old with atleast 2 examples.",
    # set temperature=0 for deterministic outputs (optional)
    temperature=0.7,
)


In [76]:
# Generate text with Gemini
# We call generate_content with the same prompt. The API returns text and safety metadata.
response = client.models.generate_content(
    model="gemini-2.5-flash",                       # choose generation model
    contents=[types.Content(parts=[types.Part(text=prompt_text)])],
    config=gen_config
)

print(response.text)

Oh, that's awesome! Cake is super yummy!

There are so many kinds of cake! Like, do you like **chocolate cake**? That's one kind! Or maybe you like a fun **vanilla cake with sprinkles**? That's another kind!


In [78]:
#Inspect safety ratings & usage metadata
# The SDK returns fine-grained safety ratings and token usage to help you audit results.
print("=== Safety & Usage metadata ===")
# prompt feedback: how the prompt rated under safety checks
if getattr(response, "prompt_feedback", None):
    print("Prompt feedback:", response.prompt_feedback)

# safety ratings for each candidate (if present)
if getattr(response, "safety_ratings", None):
    print("Safety ratings:", response.safety_ratings)
else:
    # in some SDK variants the safety info lives under response.candidates[*].safetyRatings
    for i, cand in enumerate(getattr(response, "candidates", [])):
        print(f"Candidate {i} safety:", getattr(cand, "safety_ratings", None))

# token usage metadata (helpful for cost estimation)
usage = getattr(response, "usage_metadata", None)
if usage:
    print("Usage metadata:", usage)
else:
    print("Usage metadata not present in this response object (varies by SDK version).")



=== Safety & Usage metadata ===
Candidate 0 safety: None
Usage metadata: cache_tokens_details=None cached_content_token_count=None candidates_token_count=57 candidates_tokens_details=None prompt_token_count=36 prompt_tokens_details=[ModalityTokenCount(
  modality=<MediaModality.TEXT: 'TEXT'>,
  token_count=36
)] thoughts_token_count=595 tool_use_prompt_token_count=None tool_use_prompt_tokens_details=None total_token_count=688 traffic_type=None
